In [61]:
#Setup
import numpy as np
from numpy.testing import assert_almost_equal
import scipy.integrate as integrate
from scipy.stats import norm
import matplotlib.pyplot as plt 
rng = np.random.default_rng(seed = 42)

In [64]:
### Constants
sigma1 = 0.11
sigma2 = 0.13
T = 1.7
mu = 0.13
r = 0.01
S0 = 149
time_break = 0.3
K = 188
KH = S0
n_sims = 1_000
n_timesteps = 1_000

In [63]:
### Helper Functions

### Time-varying vol

def time_varying_vol(t):
    if t > time_break:
        return sigma1 + sigma2 * (t - time_break)/(T - time_break)
    return sigma1

assert_almost_equal(np.array([time_varying_vol(0.2), time_varying_vol(0.4), time_varying_vol(T)]) ,
                    np.array([sigma1, 0.11928571428, sigma1 + sigma2]))

def integrated_vol(t):
    squared_vol = lambda x: time_varying_vol(x) ** 2
    result = integrate.quad(squared_vol, 0, t)
    return result[0]

assert_almost_equal(np.array([integrated_vol(0.3), integrated_vol(1), integrated_vol(1.7)]),
                    np.array([0.3 * 0.11 ** 2, 0.018090833333, 0.0484766666]))

### Black-Scholes

def black_scholes_call_pricing(S0,K,T,r,sigma):
    d1 = 1/(sigma * np.sqrt(T)) * (np.log(S0/K) + (r + sigma ** 2 / 2) * T)
    d2 = d1 - sigma * np.sqrt(T) 
    price =  S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    delta = norm.cdf(d1)
    gamma = norm.pdf(d1)/(S0 * sigma * np.sqrt(T))
    return price, delta, gamma

def black_scholes_put_pricing(S0,K,T,r,sigma):
    # note that C - P = S0 - K * exp(-r*T), put-call parity
    d1 = 1/(sigma * np.sqrt(T)) * (np.log(S0/K) + (r + sigma ** 2 / 2) * T)
    price =  black_scholes_call_price(S0,K,T,r,sigma) + K * np.exp(-r* T) - S0
    delta = norm.cdf(d1) - 1
    gamma = norm.pdf(d1)/(S0 * sigma * np.sqrt(T))
    return price, delta, gamma

np.testing.assert_almost_equal(np.array([black_scholes_call_pricing(100, 100, 1, 0.01, 0.2),black_scholes_put_pricing(100, 100, 1, 0.01, 0.2)]),
                                np.array([(8.43332, 0.55962, 0.01972),(7.43831, -0.44038, 0.01972)]) , decimal=5)
